# MD Simulations

Objectives
- Run a short (100 ns production) MD simulation on each mutant output from PyRosetta.

In [4]:
from pathlib import Path
from pdbfixer import PDBFixer
from openmm.app import *
from openmm import unit, LangevinMiddleIntegrator, Platform
from tqdm import trange

In [ ]:
# MD Parameters
STEP_SIZE      = 2 * unit.femtoseconds
RECORD_EVERY   = 5000                     # steps  (= 10 ps)
NVT_STEPS      = 50000                   # 100 ps
NPT_STEPS      = 50000                   # 100 ps
PROD_STEPS     = 100000                  # 200 ps
TEMPERATURE    = 300 * unit.kelvin
PRESSURE       = 1 * unit.atmosphere
NONBOND_CUTOFF = 1.0 * unit.nanometer
PADDING        = 1.0 * unit.nanometer
SEED           = 42

## Fix PDBs for simulation

In [ ]:
def fix_pdb(input_path: str | Path, output_path: str | Path | None=None):
    """
    Fix a PDB file using PDBFixer by adding missing residues, atoms, and hydrogens.

    Args:
        input_path (str or Path): Path to the input PDB file.
        output_path (str or Path, optional): Path to save the fixed PDB file. 
            If None, saves as '<input_stem>_fixed.pdb' in the same directory as input.

    Returns:
        None. Writes the fixed PDB file to the specified output path.
    """
    input_path = Path(input_path)
    output_path = Path(output_path) if output_path != None else input_path.parent / (input_path.stem + '_fixed.pdb')
    
    fixer = PDBFixer(filename=str(input_path))
    fixer.findMissingResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(pH=7.0)

    with open(output_path, 'w') as f:
        PDBFile.writeFile(fixer.topology, fixer.positions, f)

In [ ]:
def fix_all_pdb_in_directory(input_dir: str | Path, output_dir: str | Path):
    """
    Fix all PDB files in a directory using PDBFixer and save them to an output directory.

    Args:
        input_dir (str or Path): Directory containing input PDB files.
        output_dir (str or Path): Directory to save the fixed PDB files. Created if it does not exist.

    Returns:
        None. Writes fixed PDB files with '_fixed.pdb' appended to the filename in the output directory.
    """
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    pdb_files = list(input_dir.glob('*.pdb'))

    for pdb in pdb_files:
        output_path = output_dir / (pdb.stem + '_fixed.pdb')
        fix_pdb(pdb, output_path)

In [13]:
fix_all_pdb_in_directory('mutant_library', 'md_sim/inputs')